# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb690dfafd0>
├── 'a' --> tensor([[-0.9242,  0.6839, -0.9048],
│                   [ 0.8493,  0.5953,  1.4561]])
└── 'x' --> <FastTreeValue 0x7fb690e12e80>
    └── 'c' --> tensor([[-1.2699, -0.2150, -0.9804,  0.9702],
                        [-0.5845, -1.4494, -0.5616, -1.8622],
                        [-0.8930,  0.0404, -0.5812,  0.4177]])

In [4]:
t.a

tensor([[-0.9242,  0.6839, -0.9048],
        [ 0.8493,  0.5953,  1.4561]])

In [5]:
%timeit t.a

73.3 ns ± 1.84 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb690dfafd0>
├── 'a' --> tensor([[0.4780, 1.1582, 0.5800],
│                   [0.0551, 1.1295, 1.0384]])
└── 'x' --> <FastTreeValue 0x7fb690e12e80>
    └── 'c' --> tensor([[-1.2699, -0.2150, -0.9804,  0.9702],
                        [-0.5845, -1.4494, -0.5616, -1.8622],
                        [-0.8930,  0.0404, -0.5812,  0.4177]])

In [7]:
%timeit t.a = new_value

83.5 ns ± 1.22 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9242,  0.6839, -0.9048],
               [ 0.8493,  0.5953,  1.4561]]),
    x: Batch(
           c: tensor([[-1.2699, -0.2150, -0.9804,  0.9702],
                      [-0.5845, -1.4494, -0.5616, -1.8622],
                      [-0.8930,  0.0404, -0.5812,  0.4177]]),
       ),
)

In [10]:
b.a

tensor([[-0.9242,  0.6839, -0.9048],
        [ 0.8493,  0.5953,  1.4561]])

In [11]:
%timeit b.a

74.7 ns ± 1.86 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1644, -0.0450, -0.8340],
               [ 0.6624, -0.1127,  0.6565]]),
    x: Batch(
           c: tensor([[-1.2699, -0.2150, -0.9804,  0.9702],
                      [-0.5845, -1.4494, -0.5616, -1.8622],
                      [-0.8930,  0.0404, -0.5812,  0.4177]]),
       ),
)

In [13]:
%timeit b.a = new_value

638 ns ± 7.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.04 µs ± 17.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.8 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

287 µs ± 17.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

268 µs ± 5.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb5b65d0970>
├── 'a' --> tensor([[[-0.9242,  0.6839, -0.9048],
│                    [ 0.8493,  0.5953,  1.4561]],
│           
│                   [[-0.9242,  0.6839, -0.9048],
│                    [ 0.8493,  0.5953,  1.4561]],
│           
│                   [[-0.9242,  0.6839, -0.9048],
│                    [ 0.8493,  0.5953,  1.4561]],
│           
│                   [[-0.9242,  0.6839, -0.9048],
│                    [ 0.8493,  0.5953,  1.4561]],
│           
│                   [[-0.9242,  0.6839, -0.9048],
│                    [ 0.8493,  0.5953,  1.4561]],
│           
│                   [[-0.9242,  0.6839, -0.9048],
│                    [ 0.8493,  0.5953,  1.4561]],
│           
│                   [[-0.9242,  0.6839, -0.9048],
│                    [ 0.8493,  0.5953,  1.4561]],
│           
│                   [[-0.9242,  0.6839, -0.9048],
│                    [ 0.8493,  0.5953,  1.4561]]])
└── 'x' --> <FastTreeValue 0x7fb5b65d0820>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb68cd197c0>
├── 'a' --> tensor([[-0.9242,  0.6839, -0.9048],
│                   [ 0.8493,  0.5953,  1.4561],
│                   [-0.9242,  0.6839, -0.9048],
│                   [ 0.8493,  0.5953,  1.4561],
│                   [-0.9242,  0.6839, -0.9048],
│                   [ 0.8493,  0.5953,  1.4561],
│                   [-0.9242,  0.6839, -0.9048],
│                   [ 0.8493,  0.5953,  1.4561],
│                   [-0.9242,  0.6839, -0.9048],
│                   [ 0.8493,  0.5953,  1.4561],
│                   [-0.9242,  0.6839, -0.9048],
│                   [ 0.8493,  0.5953,  1.4561],
│                   [-0.9242,  0.6839, -0.9048],
│                   [ 0.8493,  0.5953,  1.4561],
│                   [-0.9242,  0.6839, -0.9048],
│                   [ 0.8493,  0.5953,  1.4561]])
└── 'x' --> <FastTreeValue 0x7fb68cd195b0>
    └── 'c' --> tensor([[-1.2699, -0.2150, -0.9804,  0.9702],
                        [-0.5845, -1.4494, -0.5616, -1.8622],
                 

In [23]:
%timeit t_cat(trees)

42.1 µs ± 906 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

93.1 µs ± 2.66 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.9242,  0.6839, -0.9048],
                [ 0.8493,  0.5953,  1.4561]],
       
               [[-0.9242,  0.6839, -0.9048],
                [ 0.8493,  0.5953,  1.4561]],
       
               [[-0.9242,  0.6839, -0.9048],
                [ 0.8493,  0.5953,  1.4561]],
       
               [[-0.9242,  0.6839, -0.9048],
                [ 0.8493,  0.5953,  1.4561]],
       
               [[-0.9242,  0.6839, -0.9048],
                [ 0.8493,  0.5953,  1.4561]],
       
               [[-0.9242,  0.6839, -0.9048],
                [ 0.8493,  0.5953,  1.4561]],
       
               [[-0.9242,  0.6839, -0.9048],
                [ 0.8493,  0.5953,  1.4561]],
       
               [[-0.9242,  0.6839, -0.9048],
                [ 0.8493,  0.5953,  1.4561]]]),
    x: Batch(
           c: tensor([[[-1.2699, -0.2150, -0.9804,  0.9702],
                       [-0.5845, -1.4494, -0.5616, -1.8622],
                       [-0.8930,  0.0404, -0.5812,  0.4177]],
         

In [26]:
%timeit Batch.stack(batches)

107 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.9242,  0.6839, -0.9048],
               [ 0.8493,  0.5953,  1.4561],
               [-0.9242,  0.6839, -0.9048],
               [ 0.8493,  0.5953,  1.4561],
               [-0.9242,  0.6839, -0.9048],
               [ 0.8493,  0.5953,  1.4561],
               [-0.9242,  0.6839, -0.9048],
               [ 0.8493,  0.5953,  1.4561],
               [-0.9242,  0.6839, -0.9048],
               [ 0.8493,  0.5953,  1.4561],
               [-0.9242,  0.6839, -0.9048],
               [ 0.8493,  0.5953,  1.4561],
               [-0.9242,  0.6839, -0.9048],
               [ 0.8493,  0.5953,  1.4561],
               [-0.9242,  0.6839, -0.9048],
               [ 0.8493,  0.5953,  1.4561]]),
    x: Batch(
           c: tensor([[-1.2699, -0.2150, -0.9804,  0.9702],
                      [-0.5845, -1.4494, -0.5616, -1.8622],
                      [-0.8930,  0.0404, -0.5812,  0.4177],
                      [-1.2699, -0.2150, -0.9804,  0.9702],
                      [-0.5845, -

In [28]:
%timeit Batch.cat(batches)

196 µs ± 3.7 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

625 µs ± 37.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
